In [ ]:
import plotly.express as px
import pandas as pd

from src import settings
from src.visualization.plot import line
from src.data.make_knn_dataset import get_organ_labels

# plot organ spectra

In [ ]:
spectra_file = settings.figures_dir / 'semantic_reflectance.csv'
df = pd.read_csv(spectra_file)
df = df[df.organ != 'gallbladder']

In [ ]:
def prepare_data(df: pd.DataFrame):
    to_replace = {
        'simulated_adapted': 'synthetic adapted',
        'simulated_sampled': 'synthetic sampled',
        'small_bowel': 'small bowel'
    }
    to_rename = {
        'wavelength': 'wavelength [nm]',
        'dataset': 'data'
    }
    new_df = df.replace(to_replace, inplace=False)
    new_df.rename(to_rename, inplace=True, axis=1)
    to_rename.update(to_replace)
    return new_df, to_rename

In [ ]:
organs = get_organ_labels()['organ_labels']
organs = [o for o in organs if o != 'gallbladder']
organs = [' '.join(o.split('_')) for o in organs]
df_prepared, mapper = prepare_data(df)
fig, plot_data = line(
    data_frame=df_prepared,
    x=mapper.get("wavelength"),
    y="reflectance",
    facet_col="organ",
    color=mapper.get("dataset"),
    facet_col_wrap=5,
    template="plotly_white",
    width=800,
    height=400,
    category_orders=dict(organ=organs),
    facet_row_spacing=0.2
)
fig.update_layout(font=dict(size=12, family="Whitney Book"))
fig.update_xaxes(title_font=dict(size=12, family="Whitney Book"))
fig.update_yaxes(title_font=dict(size=12, family="Whitney Book"))
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.write_image(settings.figures_dir / 'manuscript' / 'organ_reflectance.pdf')